# Encode/Decode SpectrumFrame payloads to/from base64 JSON

This notebook demonstrates how to encode and decode SpectrumFrame objects to/from base64 JSON format. SpectrumFrame are data structures that contain spectral information such as power spectral density (PSD), resolution bandwidth (RBW), start frequency and other parameters related to spectral analysis.

Through the following examples, we will see how to:
- Create a SpectrumFrame with example data
- Encode it to a base64 JSON string
- Decode it back to a SpectrumFrame object 
- Verify that the conversion was lossless


In [1]:
from rfscope.dsp.spectrum_codec import spectrumframe_to_b64, b64_to_spectrumframe
from rfscope.common.models import SpectrumFrame
import numpy as np

In this example we generate a SpectrumFrame object with synthetic test data, containing:

- A random power spectral density (PSD) signal generated using `np.random.normal` with 256 points
- Resolution bandwidth (RBW) of 1 Hz 
- Start frequency of 100 MHz
- 10 averages
- Noise floor of -150 dBm/Hz
- Metadata indicating this is a manual test

This simulated data will allow us to test the base64 encoding/decoding functions.


In [12]:
spectrumframe = SpectrumFrame(
    psd_dbm_per_hz=np.random.normal(size=2 ** 8),
    rbw_hz=1.0,
    f_start_hz=100e6,
    vbw_hz=None,
    window=None,
    averages=10,
    noise_floor_dbm_per_hz=-150.0,
    metadata={"source": "manual-test"},
)
spectrumframe

SpectrumFrame(psd_dbm_per_hz=array([-0.39810591,  0.44584921, -1.07476901,  0.19258734,  0.58877667,
        0.70602828, -1.45842308, -2.59885666,  0.26371824,  0.01704271,
        0.17900362, -0.9992248 , -0.32436327, -0.3823979 ,  0.38912887,
        0.65460093,  0.05101213,  0.53927261,  1.81942079,  0.69452108,
       -0.08237735, -0.35463551,  1.41394101,  0.61666402, -0.31588082,
        0.67177774, -1.33901183,  0.83012766, -1.25052346, -0.96308857,
        1.45045218, -0.24191703,  0.44921357,  1.60573478,  0.04293111,
        0.99843316,  0.37792243,  0.29230699,  1.45018173, -0.24626616,
       -0.32929642,  0.52713124,  0.66485291, -1.35337427,  0.12471062,
        0.0942092 ,  0.20116989, -0.59883989, -1.4224333 ,  3.55887876,
        0.10754161,  0.39683387,  0.46227641, -0.79520364,  1.26108076,
       -1.05550119,  0.58806897, -0.75946944,  0.32232615,  1.16898717,
       -2.05375654,  1.64724958,  0.57582508, -0.96680925, -1.34701274,
       -0.94231076,  0.75934328, -0

The `spectrumframe_to_b64` function converts a SpectrumFrame object into a base64-encoded JSON string. It:

1. Accepts a SpectrumFrame object as input, which contains spectral data like PSD values, RBW, start frequency, etc.
2. Takes optional parameters:
   - level: Compression level (1-9) for the zlib algorithm
   - dtype: NumPy data type for PSD values (e.g. np.float32)
3. Compresses the PSD data array using zlib
4. Creates a JSON object with the frame's metadata and compressed PSD
5. Encodes the JSON as base64
6. Returns the encoded string for efficient data transmission or storage

This encoding allows SpectrumFrame objects to be serialized and later reconstructed using `b64_to_spectrumframe`.


In [ ]:
b64 = spectrumframe_to_b64(spectrumframe, level=6, dtype=np.float32)
b64

The `b64_to_spectrumframe` function converts a base64-encoded JSON string back into a SpectrumFrame object. It performs these steps:

1. Takes a base64-encoded string as input that contains compressed spectral data and metadata
2. Decodes the base64 string back to a JSON object
3. Decompresses the PSD data array using zlib decompression
4. Reconstructs a complete SpectrumFrame object with:
   - Power Spectral Density (PSD) values
   - Resolution bandwidth (RBW)
   - Start frequency
   - Other parameters like VBW, window type, averages
   - Metadata

This function is the reverse operation of `spectrumframe_to_b64`, allowing spectral data to be reconstructed after transmission or storage.


In [10]:
spectrumframe_dec = b64_to_spectrumframe(b64)
spectrumframe_dec

SpectrumFrame(psd_dbm_per_hz=array([-0.38123161,  0.56174672,  0.40798554, -0.64209402, -1.60040081,
        0.1443435 ,  0.92721075, -0.86940002,  0.6069656 , -1.82786465,
       -0.34421599, -1.52997243,  0.5353874 , -0.01034733,  0.08799654,
        1.16870332, -2.41259909, -1.55795944,  1.18343556, -0.18668991,
       -0.31675315,  0.83310229,  0.69673347, -0.33185685,  1.79961002,
        0.8883273 ,  1.22740507,  0.96074748, -0.27207404,  0.22959007,
       -0.39596161, -2.30852985,  0.07097191, -0.29879045,  1.04978061,
        0.17316896,  0.47456503,  2.34408832,  0.64188343,  0.6265347 ,
        0.25362152, -0.5640837 , -0.14194998, -0.21708123, -0.58840615,
       -0.02774872, -0.03094758,  0.10428508,  0.91558433, -0.23687685,
        0.93539006,  0.34020093, -0.79414374, -0.59478641, -0.11436195,
       -0.81786925, -0.20094211, -0.73166072, -2.25396705, -0.18276757,
       -1.93241739,  0.57434303,  0.34563962, -1.15695095, -0.22824711,
        0.78610629, -0.7141242 , -1

The comparison `spectrumframe == spectrumframe_dec` verifies if the original SpectrumFrame object is equal to the decoded SpectrumFrame object after the base64 encoding and decoding process. This validates that the conversion was lossless and all the data (PSD values, RBW, start frequency, metadata, etc.) was preserved correctly during the encoding and decoding operations.


In [11]:
spectrumframe == spectrumframe_dec

True